# Fix Python Issue - I

I am running this on a newly installed SQL Server 2019 CU4, with MS R & Python installed.


In [1]:
USE Master;
GO

DROP DATABASE IF EXISTS ExtLangDB ;
GO

CREATE DATABASE ExtLangDB;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.278

In [2]:
USE ExtLangDB;
GO

Commands completed successfully.

Total execution time: 00:00:00

In [13]:
SELECT DB_NAME();

(1 row affected)

Total execution time: 00:00:00.011

(No column name)
ExtLangDB


In [14]:
SELECT * FROM sys.EXTERNAL_LANGUAGES

(3 rows affected)

Total execution time: 00:00:00.011

external_language_id,language,create_date,principal_id
1,R,2020-12-27 05:42:57.890,4
2,Python,2020-12-27 05:42:57.890,4
65537,p37dbg,2021-01-14 05:09:35.570,1


Let's check the version of Python.

In [1]:
EXEC sp_execute_external_script
@language =N'Python',
@script=N'
import pandas as pd
import sys
df = pd.DataFrame(columns=["Version"])
ver = sys.version
pth = sys.executable
df = df.append({"Version": ver}, ignore_index=True)
OutputDataSet = df'
WITH RESULT SETS (([Python Version] nvarchar(256)));

(1 row affected)

Total execution time: 00:00:08.144

Python Version
"3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]"


In [40]:
DROP EXTERNAL LANGUAGE p37dbg

: Msg 15151, Level 16, State 2, Line 1
Cannot drop the external language 'p37dbg', because it does not exist or you do not have permission.

Total execution time: 00:00:00

Create external language with the Microsoft Python extension, (supports Python 3.7.x)

In [41]:
USE ExtLangDB;
GO

CREATE EXTERNAL LANGUAGE p37dbg
FROM 
(
  CONTENT = 'W:\OneDrive\sql-ext-lang\python\python-lang-extension-windows-debug-3.7.zip'
  , FILE_NAME = 'pythonextension.dll'
  , ENVIRONMENT_VARIABLES = N'{"PYTHONHOME":"C:\\Python37"}'
);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.002

In [42]:
SELECT * FROM sys.external_languages;

(3 rows affected)

Total execution time: 00:00:00.012

external_language_id,language,create_date,principal_id
1,R,2020-12-27 05:42:57.890,4
2,Python,2020-12-27 05:42:57.890,4
65541,p37dbg,2021-01-14 06:03:29.443,1


Restart the launchpad service.

Use Python 3.7

In [44]:
EXEC sp_execute_external_script
@language =N'p37dbg',
@script=N'
import pandas as pd
import sys
df = pd.DataFrame(columns=["Version"])
ver = sys.version
pth = sys.executable
df = df.append({"Version": ver}, ignore_index=True)
OutputDataSet = df'
WITH RESULT SETS (([Python Version] nvarchar(256)));

Total execution time: 00:00:00.214

Python Version


: Msg 39004, Level 16, State 20, Line 0
A 'p37dbg' script error occurred during execution of 'sp_execute_external_script' with HRESULT 0x80004004.

Create Python 3.9 language

In [13]:
USE ExtLangDB;
GO

CREATE EXTERNAL LANGUAGE p39
FROM (CONTENT = 'W:\python-lang-extension-windows-3.9.zip'
      , FILE_NAME = 'pythonextension.dll');
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.002

Before executing the code below, restart the Launchpad service.

In [16]:
EXEC sp_execute_external_script
@language =N'p39',
@script=N'
import pandas as pd
import sys
df = pd.DataFrame(columns=["Version"])
ver = sys.version
df = df.append({"Version": ver}, ignore_index=True)
OutputDataSet = df'
WITH RESULT SETS (([Python Version] nvarchar(256)));

STDOUT message(s) from external script:

(1 row affected)

Total execution time: 00:00:00.523

Python Version
"3.9.1 (tags/v3.9.1:1e5d33e, Dec 7 2020, 17:08:21) [MSC v.1927 64 bit (AMD64)]"


In [7]:
DROP EXTERNAL LANGUAGE p39;

Commands completed successfully.

Total execution time: 00:00:00.002

Now go back and run the code with Python

In [28]:
EXEC sp_execute_external_script
@language =N'p37',
@script=N'
import pandas as pd
import sys
df = pd.DataFrame(columns=["Version"])
ver = sys.version
pth = sys.executable
df = df.append({"Version": ver}, ignore_index=True)
OutputDataSet = df'
WITH RESULT SETS (([Python Version] nvarchar(256)));

STDOUT message(s) from external script: 
Fatal Python error: init_sys_streams: can't initialize sys standard streams
Traceback (most recent call last):
 File "C:\Python39\lib\io.py", line 54, in 
ImportError: cannot import name 'open_code' from 'io' (unknown location)

Total execution time: 00:00:00.271

Python Version


: Msg 39004, Level 16, State 20, Line 0
A 'p37' script error occurred during execution of 'sp_execute_external_script' with HRESULT 0x800704d4.